In [52]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import os

In [53]:
os.cpu_count()

2

In [54]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

In [55]:
train_data = torchvision.datasets.CIFAR10(
    root = './data', train=True, download=True, transform=transform
)

test_data = torchvision.datasets.CIFAR10(
    root = './data', train=False, download=True, transform=transform
)

Files already downloaded and verified
Files already downloaded and verified


In [56]:
train_loader = torch.utils.data.DataLoader(
    train_data, batch_size=64, shuffle=True, num_workers=2
)

test_loader = torch.utils.data.DataLoader(
    test_data, batch_size=64, shuffle=True, num_workers=2
)

In [57]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64*8*8, 512)
        self.fc2 = nn.Linear(512, 10)


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64*8*8)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [58]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [59]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch + 1}, Loss: {running_loss / len(train_loader):.4f}')

In [60]:
train_model(model, train_loader, criterion, optimizer)

Epoch 1, Loss: 1.7370
Epoch 2, Loss: 1.4641
Epoch 3, Loss: 1.3568
Epoch 4, Loss: 1.2728
Epoch 5, Loss: 1.2154
Epoch 6, Loss: 1.1623
Epoch 7, Loss: 1.1196
Epoch 8, Loss: 1.0824
Epoch 9, Loss: 1.0512
Epoch 10, Loss: 1.0281
